SQL - Connector 

In [12]:
import pandas as pd
import os

In [26]:
import mysql.connector

connection = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "oKkfDp6Bs76GVTg.root",
  password = "jlzngSYu6mO3vwih",
  database = "Stock_analysis_50"
)
mycursor = connection.cursor(buffered=True)

In [ ]:
mycursor.execute("CREATE DATABASE IF NOT EXISTS Stock_analysis")
print("successful")


Database created or already exists.


In [6]:
mycursor.execute("CREATE DATABASE IF NOT EXISTS Stock_analysis_50")
print("successful")

successful


importing a file

In [11]:
csv_file_path = r"C:\prj_stock_analysis\datas\correlation\correlation_matrix.csv" 
df = pd.read_csv(csv_file_path)

columns_with_types = ", ".join([f"`{col}` TEXT" for col in df.columns])
create_table_query = f"CREATE TABLE IF NOT EXISTS Stock_analysis.Correlation_1_matrix ({columns_with_types})"
mycursor.execute(create_table_query)

placeholders = ", ".join(["%s"] * len(df.columns))
columns = ", ".join([f"`{col}`" for col in df.columns])
insert_query = f"INSERT INTO Correlation_1_matrix ({columns}) VALUES ({placeholders})"

mycursor.executemany(insert_query, df.values.tolist())
connection.commit()

print("successfully imported")


successfully imported


importing 50 csv's

In [ ]:


folder_path = r"C:\prj_stock_analysis\datas\aggregated_data_1"

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        filepath = os.path.join(folder_path, filename)
        table_name = os.path.splitext(filename)[0]


    print(f"\nProcessing file: {filename}")
    print("DataFrame Columns:", df.columns)
    print("DataFrame Data Types:", df.dtypes)

    columns_with_types = ", ".join([f"`{col}` TEXT" for col in df.columns])
    create_table_query = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({columns_with_types})"
    mycursor.execute(create_table_query)

 
    mycursor.execute(f"DESCRIBE `{table_name}`")
    table_columns = mycursor.fetchall()
    db_columns = [col[0] for col in table_columns]
    print("Table Columns from MySQL:", db_columns)

    
    if sorted(df.columns.tolist()) != sorted(db_columns):
        print(f"Warning: Columns don't match between DataFrame and MySQL table in {filename}.")
        print("DataFrame Columns:", df.columns)
        print("Table Columns:", db_columns)
        continue  

   
    print("\nChecking for missing values in DataFrame:")
    print(df.isnull().sum())  

    
    df = df.dropna() 

    
    placeholders = ", ".join(["%s"] * len(df.columns))
    columns_str = ", ".join([f"`{col}`" for col in df.columns])
    insert_query = f"INSERT INTO `{table_name}` ({columns_str}) VALUES ({placeholders})"

    
    try:
        
        mycursor.executemany(insert_query, df.values.tolist())
        print(f"Successfully uploaded {filename} into table {table_name}!")
    except mysql.connector.Error as err:
        print(f"Error inserting {filename}: {err}")
        connection.rollback()  


connection.commit()
mycursor.close()
connection.close()
print("All files processed successfully.")